# Interactive Primordial Soup simulation on Quantum ESPRESSO

In order to assess the Jupyter-workflow capabilities to enable interactive simulations of realistic, large-scale systems, we implement a Notebook describing a multi-step simulation workflow in Quantum ESPRESSO.

In particular, the analyzed workflow implements a Car-Parrinello simulation of a mixture of water, ammonia and methane molecules to represent the basic ingredients of life (the so-called primordial soup). The simulation aims to explore the phase space to find where C-H, O-H and N-H bonds break up, forming more complex organic molecules. Several Car-Parrinello simulations at different pressure-temperature points (P, T) are needed to simulate the phase diagram.

In [0]:
temps=['t1', 't2']
press=['p1', 'p2']

The workflow is composed of six steps, which were executed on top of the PBS-managed davinci-1 facility, the HPC centre of the Leonard S.p.A. company.

This time we did not use a Singularity container, but ran the steps directly on bare metal to fully exploit compile-time optimisations of a Quantum ESPRESSO executable compiled with Intel OneAPI. Nevertheless, we packed both the executable (named cp-oneapi.x) and all the input files inside the INPDIR directory.

In order to replicate the experiment in a fully-optimised execution environment, you need to compile the Quantum ESPRESSO cp executable directly on your facility. The node-details.txt file provides some information on the libraries used to compile Quantum ESPRESSO. Then, you need to set the correct credentials in the Notebook metadata (e.g. username and sshKey) and, if necessary, modify the PBS script template at environment/pbs_espresso.

Moreover, please note that both the input directory and the output directory should preserve their names (i.e., INPDIR and OUTDIR, respectively) because their paths are directly encoded inside Quantum ESPRESSO input files. Data dependencies are propagated between subsequent steps through the OUTDIR directory

To run all cells in the standard Jupyter interactive mode, you can use the Cell > Run All menu item, while to run in bulk mode you have to press the <button class="btn btn-default"><i class="fa-random fa"></i></button> button in the main toolbar.

The first four steps, common to all simulations, prepare a starting state at room temperature and pressure from a random distribution of the three molecules

In [0]:
import time
start = time.time()

In [ ]:
%%bash -s "$input_dir"
cp -r $1 .
mkdir OUTDIR
mpirun -np 32 -f $PBS_NODEFILE sh -c "./INPDIR/cp-oneapi.x < ./INPDIR/chno.in.00 > ./OUTDIR/chno.out.00"

In [ ]:
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
%%bash -s "$input_dir" "$output_dir"
cp -r $1 $2 .
mkdir OUTDIR
mpirun -np 32 -f $PBS_NODEFILE sh -c "./INPDIR/cp-oneapi.x < ./INPDIR/chno.in.01 > ./OUTDIR/chno.out.01"

In [ ]:
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
%%bash -s "$input_dir" "$output_dir"
cp -r $1 $2 .
mkdir OUTDIR
mpirun -np 32 -f $PBS_NODEFILE sh -c "./INPDIR/cp-oneapi.x < ./INPDIR/chno.in.02 > ./OUTDIR/chno.out.02"

In [ ]:
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
%%bash -s "$input_dir" "$output_dir"
cp -r $1 $2 .
mkdir OUTDIR
mpirun -np 32 -f $PBS_NODEFILE sh -c "./INPDIR/cp-oneapi.x < ./INPDIR/chno.in.03 > ./OUTDIR/chno.out.03"

In [ ]:
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Then the pipeline forks to simulate different temperatures through Nos&eacute;-Hoover thermostats.

In [0]:
%%bash -s "$input_dir" "$output_dir" "$temps"
TEMPS=($(echo $3 | tr -d "[],'"))
cp -r $1 .
for TEMP in ${TEMPS[*]}; do
  cp -r $2 "OUTDIR.${TEMP}"
  mpirun -np 32 -f $PBS_NODEFILE sh -c "./INPDIR/cp-oneapi.x < ./INPDIR/chno.in.04.${TEMP} > ./OUTDIR.${TEMP}/chno.out.04.${TEMP}"
done

In [0]:
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

00:00:06.84

Finally, for each temperature, the simulation forks again to simulate each temperature at several values of pressure using the Parrinello-Rahman constant pressure Lagrangian.

In [0]:
%%bash -s "$input_dir" "$temps" "$temp_outdirs" "$press"
TEMPS=($(echo $2 | tr -d "[],'"))
TEMP_OUTDIRS=($(echo $3 | tr -d "[],'"))
PRESS=($(echo $4 | tr -d "[],'"))

cp -r $1 .
for TEMP_OUTDIR in ${TEMP_OUTDIRS[*]}; do
  echo $TEMP_OUTDIRS
  cp -r "${TEMP_OUTDIR}" .
done

for TEMP in ${TEMPS[*]}; do
  for PRS in ${PRESS[*]}; do
    mkdir "OUTDIR.${TEMP}.${PRS}"
    cp -r -P ./OUTDIR.${TEMP}/*.save "OUTDIR.${TEMP}.${PRS}"
    mpirun -np 32 -f $PBS_NODEFILE sh -c "./INPDIR/cp-oneapi.x < ./INPDIR/chno.in.05.${TEMP}.${PRS} > ./OUTDIR.${TEMP}/chno.out.05.${TEMP}.${PRS}"
  done
done

/tmp/streamflow/CYQoOU/OUTDIR.t1
/tmp/streamflow/KZLQEy/OUTDIR.t1
/tmp/streamflow/LvDnAo/OUTDIR.t2
/tmp/streamflow/wNHoma/OUTDIR.t2

In [0]:
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

00:00:12.28